In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
train = pd.read_csv('train_0irEZ2H.csv')
test = pd.read_csv('test_nfaJ3J5.csv')

In [3]:
train.head()

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20
1,2,17/01/11,8091,216419,99.0375,99.0375,0,0,28
2,3,17/01/11,8091,216425,133.9500,133.9500,0,0,19
3,4,17/01/11,8091,216233,133.9500,133.9500,0,0,44
4,5,17/01/11,8091,217390,141.0750,141.0750,0,0,52


In [4]:
train.shape, test.shape

((150150, 9), (13860, 8))

In [5]:
train.dtypes

record_ID            int64
week                object
store_id             int64
sku_id               int64
total_price        float64
base_price         float64
is_featured_sku      int64
is_display_sku       int64
units_sold           int64
dtype: object

In [6]:
combine = train.append(test)
combine.shape

C:\Users\hungu\anaconda.x\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(164010, 9)

In [7]:
combine['week'] = pd.to_datetime(combine['week'], format='%d/%m/%y')
combine['Year'] = combine.week.dt.year
combine['Month'] = combine.week.dt.month
combine['Day'] = combine.week.dt.day
combine['Dayofweek'] = combine.week.dt.dayofweek

def get_weekend(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0
combine['IsWeekend'] = combine['week'].apply(get_weekend)

print(combine.columns)

combine.head()

Index(['base_price', 'is_display_sku', 'is_featured_sku', 'record_ID',
       'sku_id', 'store_id', 'total_price', 'units_sold', 'week', 'Year',
       'Month', 'Day', 'Dayofweek', 'IsWeekend'],
      dtype='object')


,base_price,is_display_sku,is_featured_sku,record_ID,sku_id,store_id,total_price,units_sold,week,Year,Month,Day,Dayofweek,IsWeekend
0,111.8625,0,0,1,216418,8091,99.0375,20.0,2011-01-17,2011,1,17,0,0
1,99.0375,0,0,2,216419,8091,99.0375,28.0,2011-01-17,2011,1,17,0,0
2,133.9500,0,0,3,216425,8091,133.9500,19.0,2011-01-17,2011,1,17,0,0
3,133.9500,0,0,4,216233,8091,133.9500,44.0,2011-01-17,2011,1,17,0,0
4,141.0750,0,0,5,217390,8091,141.0750,52.0,2011-01-17,2011,1,17,0,0


In [8]:
combine['Year'].value_counts()

2012    60060
2011    57750
2013    46200
Name: Year, dtype: int64

In [9]:
combine['Month'].value_counts()

7     16170
5     16170
8     15015
4     15015
1     15015
9     13860
6     13860
3     13860
2     13860
10    12705
12     9240
11     9240
Name: Month, dtype: int64

In [10]:
combine['Day'].value_counts()

16    5775
7     5775
9     5775
5     5775
11    5775
12    5775
13    5775
14    5775
4     5775
30    5775
17    5775
18    5775
19    5775
20    5775
21    5775
23    5775
24    5775
25    5775
26    5775
27    5775
28    5775
2     5775
6     5775
3     4620
1     4620
10    4620
8     3465
15    3465
22    3465
29    3465
31    3465
Name: Day, dtype: int64

In [11]:
combine['Dayofweek'].value_counts()

1    95865
0    68145
Name: Dayofweek, dtype: int64

In [12]:
combine['IsWeekend'].value_counts()

0    164010
Name: IsWeekend, dtype: int64

In [13]:
combine = combine.drop('IsWeekend', axis=1)
combine.shape

(164010, 13)

In [14]:
combine['discount_price'] = combine['base_price'] - combine['total_price']
combine.head()

,base_price,is_display_sku,is_featured_sku,record_ID,sku_id,store_id,total_price,units_sold,week,Year,Month,Day,Dayofweek,discount_price
0,111.8625,0,0,1,216418,8091,99.0375,20.0,2011-01-17,2011,1,17,0,12.825
1,99.0375,0,0,2,216419,8091,99.0375,28.0,2011-01-17,2011,1,17,0,0.000
2,133.9500,0,0,3,216425,8091,133.9500,19.0,2011-01-17,2011,1,17,0,0.000
3,133.9500,0,0,4,216233,8091,133.9500,44.0,2011-01-17,2011,1,17,0,0.000
4,141.0750,0,0,5,217390,8091,141.0750,52.0,2011-01-17,2011,1,17,0,0.000


In [15]:
operation_mean = ['mean']
operation_sum_mean = ['sum', 'mean']

bb_op = {'total_price': operation_mean, 
         'base_price': operation_mean, 
         'discount_price': operation_sum_mean}

num_grouping = combine.groupby(['Day', 'store_id', 'sku_id'])
num_groupby = num_grouping.agg(bb_op)
num_groupby.head()

total_price  base_price discount_price          
                           mean        mean            sum      mean
Day store_id sku_id                                                 
1   8023     216233  128.071875  131.812500        14.9625  3.740625
             216418  102.778125  110.615625        31.3500  7.837500
             216419  103.134375  110.437500        29.2125  7.303125
             216425  126.646875  130.031250        13.5375  3.384375
             217390  147.843750  154.434375        26.3625  6.590625

In [16]:
num_groupby.columns = ['n_' + '_'.join(col).strip() for col in num_groupby.columns.values]
num_groupby.reset_index(inplace=True)
num_groupby.head()

,Day,store_id,sku_id,n_total_price_mean,n_base_price_mean,n_discount_price_sum,n_discount_price_mean
0,1,8023,216233,128.071875,131.812500,14.9625,3.740625
1,1,8023,216418,102.778125,110.615625,31.3500,7.837500
2,1,8023,216419,103.134375,110.437500,29.2125,7.303125
3,1,8023,216425,126.646875,130.031250,13.5375,3.384375
4,1,8023,217390,147.843750,154.434375,26.3625,6.590625


In [17]:
num_groupby.shape

(35805, 7)

In [18]:
combine = pd.merge(combine, num_groupby)
combine.head()

,base_price,is_display_sku,is_featured_sku,record_ID,sku_id,store_id,total_price,units_sold,week,Year,Month,Day,Dayofweek,discount_price,n_total_price_mean,n_base_price_mean,n_discount_price_sum,n_discount_price_mean
0,111.8625,0,0,1,216418,8091,99.0375,20.0,2011-01-17,2011,1,17,0,12.825,104.7375,107.3025,12.825,2.565
1,104.0250,0,0,63683,216418,8091,104.0250,33.0,2011-10-17,2011,10,17,0,0.000,104.7375,107.3025,12.825,2.565
2,104.0250,0,0,106227,216418,8091,104.0250,40.0,2012-04-17,2012,4,17,1,0.000,104.7375,107.3025,12.825,2.565
3,106.1625,0,0,127509,216418,8091,106.1625,31.0,2012-07-17,2012,7,17,1,0.000,104.7375,107.3025,12.825,2.565
4,110.4375,0,0,227368,216418,8091,110.4375,NaN,2013-09-17,2013,9,17,1,0.000,104.7375,107.3025,12.825,2.565


In [19]:
X = combine[combine['units_sold'].isnull()!=True].drop(['week', 'record_ID', 'sku_id', 'store_id','units_sold'], axis=1)
y = np.log(combine[combine['units_sold'].isnull()!=True]['units_sold'].reset_index(drop=True))

X_test = combine[combine['units_sold'].isnull()==True].drop(['week', 'record_ID', 'sku_id', 'store_id', 'units_sold'], axis=1)

X.shape, y.shape, X_test.shape

((150150, 13), (150150,), (13860, 13))

In [20]:
X.head()

,base_price,is_display_sku,is_featured_sku,total_price,Year,Month,Day,Dayofweek,discount_price,n_total_price_mean,n_base_price_mean,n_discount_price_sum,n_discount_price_mean
0,111.8625,0,0,99.0375,2011,1,17,0,12.825,104.7375,107.3025,12.825,2.565
1,104.0250,0,0,104.0250,2011,10,17,0,0.000,104.7375,107.3025,12.825,2.565
2,104.0250,0,0,104.0250,2012,4,17,1,0.000,104.7375,107.3025,12.825,2.565
3,106.1625,0,0,106.1625,2012,7,17,1,0.000,104.7375,107.3025,12.825,2.565
5,99.0375,0,0,99.0375,2011,1,17,0,0.000,105.4500,105.4500,0.000,0.000


In [21]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold

In [22]:
err = []
y_pred_tot_lgm = []
features = X.columns
feature_importance_df = pd.DataFrame()

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=200)
    pred_y = m.predict(x_val)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print(i, "err_lgm: ",np.sqrt(mean_squared_log_error(pred_y, y_val)))
    err.append(np.sqrt(mean_squared_log_error(pred_y, y_val)))
    pred_test = m.predict(X_test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.640393	training's l2: 0.410103	valid_1's rmse: 0.647432	valid_1's l2: 0.419168
[400]	training's rmse: 0.617003	training's l2: 0.380692	valid_1's rmse: 0.636906	valid_1's l2: 0.40565
[600]	training's rmse: 0.603387	training's l2: 0.364076	valid_1's rmse: 0.633684	valid_1's l2: 0.401555
[800]	training's rmse: 0.59356	training's l2: 0.352314	valid_1's rmse: 0.632663	valid_1's l2: 0.400263
[1000]	training's rmse: 0.586278	training's l2: 0.343722	valid_1's rmse: 0.631435	valid_1's l2: 0.39871
[1200]	training's rmse: 0.580033	training's l2: 0.336438	valid_1's rmse: 0.630004	valid_1's l2: 0.396905
[1400]	training's rmse: 0.574056	training's l2: 0.32954	valid_1's rmse: 0.629743	valid_1's l2: 0.396577
[1600]	training's rmse: 0.569266	training's l2: 0.324064	valid_1's rmse: 0.629379	valid_1's l2: 0.396118
[1800]	training's rmse: 0.565149	training's l2: 0.319394	valid_1's rmse: 0.629041	valid_1's l2: 0.395692
E

Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.641341	training's l2: 0.411318	valid_1's rmse: 0.648115	valid_1's l2: 0.420053
[400]	training's rmse: 0.61807	training's l2: 0.382011	valid_1's rmse: 0.636469	valid_1's l2: 0.405092
[600]	training's rmse: 0.60428	training's l2: 0.365155	valid_1's rmse: 0.633334	valid_1's l2: 0.401111
[800]	training's rmse: 0.594715	training's l2: 0.353686	valid_1's rmse: 0.631288	valid_1's l2: 0.398524
[1000]	training's rmse: 0.587172	training's l2: 0.344771	valid_1's rmse: 0.630218	valid_1's l2: 0.397175
[1200]	training's rmse: 0.580496	training's l2: 0.336975	valid_1's rmse: 0.629305	valid_1's l2: 0.396025
[1400]	training's rmse: 0.574368	training's l2: 0.329898	valid_1's rmse: 0.628847	valid_1's l2: 0.395449
[1600]	training's rmse: 0.569845	training's l2: 0.324723	valid_1's rmse: 0.627999	valid_1's l2: 0.394383
[1800]	training's rmse: 0.565242	training's l2: 0.319499	valid_1's rmse: 0.627587	valid_1's l2: 0.393866

[1800]	training's rmse: 0.565683	training's l2: 0.319997	valid_1's rmse: 0.63172	valid_1's l2: 0.39907
[2000]	training's rmse: 0.561396	training's l2: 0.315166	valid_1's rmse: 0.631165	valid_1's l2: 0.398369
[2200]	training's rmse: 0.55774	training's l2: 0.311074	valid_1's rmse: 0.630415	valid_1's l2: 0.397423
[2400]	training's rmse: 0.554328	training's l2: 0.30728	valid_1's rmse: 0.629489	valid_1's l2: 0.396256
[2600]	training's rmse: 0.550997	training's l2: 0.303598	valid_1's rmse: 0.629148	valid_1's l2: 0.395827
[2800]	training's rmse: 0.548147	training's l2: 0.300465	valid_1's rmse: 0.628896	valid_1's l2: 0.39551
Early stopping, best iteration is:
[2695]	training's rmse: 0.549695	training's l2: 0.302164	valid_1's rmse: 0.628675	valid_1's l2: 0.395233
14 err_lgm:  0.16113234071952381
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.639712	training's l2: 0.409232	valid_1's rmse: 0.667778	valid_1's l2: 0.445927
[400]	training's rmse: 0.616399	trai

In [23]:
np.mean(err,0)

0.16841952498756144

In [24]:
all_feat = feature_importance_df[["Feature", 
                                      "importance"]].groupby("Feature").mean().sort_values(by="importance", 
                                                                                           ascending=False)
all_feat.reset_index(inplace=True)
important_feat = list(all_feat['Feature'])
all_feat

,Feature,importance
0,n_base_price_mean,6917.666667
1,n_total_price_mean,6253.733333
2,base_price,6011.733333
3,total_price,5624.533333
4,n_discount_price_sum,5613.266667
5,Day,4986.466667
6,n_discount_price_mean,4596.533333
7,Month,3392.666667
8,discount_price,2817.133333
9,Year,1329.533333


In [25]:
df = X[important_feat]
corr_matrix = df.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

high_cor = [column for column in upper.columns if any(upper[column] > 0.98)]
print(len(high_cor))
print(high_cor)

3
['n_total_price_mean', 'base_price', 'n_discount_price_mean']


In [26]:
features = [i for i in important_feat if i not in high_cor]
print(len(features))
print(features)

10
['n_base_price_mean', 'total_price', 'n_discount_price_sum', 'Day', 'Month', 'discount_price', 'Year', 'is_display_sku', 'Dayofweek', 'is_featured_sku']


In [27]:
X1 = X[features]
X_test1 = X_test[features]

In [28]:
err = []
y_pred_tot_lgm_1 = []

from sklearn.model_selection import KFold

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X1, y):
    x_train, x_val = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=200)
    pred_y = m.predict(x_val)
    
    print("err_lgm: ",np.sqrt(mean_squared_log_error(pred_y, y_val)))
    err.append(np.sqrt(mean_squared_log_error(pred_y, y_val)))
    pred_test = m.predict(X_test1)
    i = i + 1
    y_pred_tot_lgm_1.append(pred_test)

Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.650427	training's l2: 0.423056	valid_1's rmse: 0.65281	valid_1's l2: 0.426161
[400]	training's rmse: 0.627609	training's l2: 0.393893	valid_1's rmse: 0.64016	valid_1's l2: 0.409805
[600]	training's rmse: 0.615622	training's l2: 0.37899	valid_1's rmse: 0.636922	valid_1's l2: 0.40567
[800]	training's rmse: 0.606789	training's l2: 0.368192	valid_1's rmse: 0.634681	valid_1's l2: 0.40282
[1000]	training's rmse: 0.60042	training's l2: 0.360504	valid_1's rmse: 0.633173	valid_1's l2: 0.400908
[1200]	training's rmse: 0.594472	training's l2: 0.353397	valid_1's rmse: 0.632369	valid_1's l2: 0.39989
[1400]	training's rmse: 0.589805	training's l2: 0.34787	valid_1's rmse: 0.632209	valid_1's l2: 0.399688
[1600]	training's rmse: 0.585471	training's l2: 0.342776	valid_1's rmse: 0.632016	valid_1's l2: 0.399444
Early stopping, best iteration is:
[1529]	training's rmse: 0.586925	training's l2: 0.344481	valid_1's rmse: 0.

[1200]	training's rmse: 0.595189	training's l2: 0.35425	valid_1's rmse: 0.638361	valid_1's l2: 0.407504
[1400]	training's rmse: 0.590908	training's l2: 0.349172	valid_1's rmse: 0.637578	valid_1's l2: 0.406506
[1600]	training's rmse: 0.586723	training's l2: 0.344244	valid_1's rmse: 0.636723	valid_1's l2: 0.405416
[1800]	training's rmse: 0.582982	training's l2: 0.339868	valid_1's rmse: 0.635155	valid_1's l2: 0.403422
[2000]	training's rmse: 0.578933	training's l2: 0.335163	valid_1's rmse: 0.634431	valid_1's l2: 0.402503
[2200]	training's rmse: 0.575273	training's l2: 0.330939	valid_1's rmse: 0.633899	valid_1's l2: 0.401827
[2400]	training's rmse: 0.571461	training's l2: 0.326568	valid_1's rmse: 0.633006	valid_1's l2: 0.400697
[2600]	training's rmse: 0.568685	training's l2: 0.323403	valid_1's rmse: 0.632601	valid_1's l2: 0.400184
[2800]	training's rmse: 0.566336	training's l2: 0.320737	valid_1's rmse: 0.632315	valid_1's l2: 0.399823
[3000]	training's rmse: 0.56363	training's l2: 0.317679	

[2000]	training's rmse: 0.575882	training's l2: 0.33164	valid_1's rmse: 0.669501	valid_1's l2: 0.448231
[2200]	training's rmse: 0.572553	training's l2: 0.327817	valid_1's rmse: 0.669283	valid_1's l2: 0.447939
Early stopping, best iteration is:
[2052]	training's rmse: 0.574962	training's l2: 0.330581	valid_1's rmse: 0.668984	valid_1's l2: 0.447539
err_lgm:  0.17573827907163125
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.650311	training's l2: 0.422905	valid_1's rmse: 0.668927	valid_1's l2: 0.447464
[400]	training's rmse: 0.628128	training's l2: 0.394545	valid_1's rmse: 0.655139	valid_1's l2: 0.429207
[600]	training's rmse: 0.61664	training's l2: 0.380245	valid_1's rmse: 0.649279	valid_1's l2: 0.421564
[800]	training's rmse: 0.608152	training's l2: 0.369849	valid_1's rmse: 0.645459	valid_1's l2: 0.416618
[1000]	training's rmse: 0.60094	training's l2: 0.361129	valid_1's rmse: 0.642383	valid_1's l2: 0.412656
[1200]	training's rmse: 0.595213	trainin

In [29]:
np.mean(err,0)

0.16967383458460722

In [30]:
submission = pd.DataFrame()
submission['record_ID'] = test['record_ID']
submission['units_sold'] = np.exp(np.mean(y_pred_tot_lgm, 0)).astype('int')
submission.head()

,record_ID,units_sold
0,212645,36
1,212646,37
2,212647,35
3,212648,45
4,212649,57


In [31]:
submission.to_csv('submission.csv', index=False)